# Vector Similarity Astra-Bedrock Search QA Quickstart

Set up a simple Question-Answering system with LangChain and Amazon Bedrock, using Astra DB as the Vector Database.

## Prerequisites

Make sure you have a vector-capable Astra database (get one for free at [astra.datastax.com](https://astra.datastax.com)):

- You will be asked to provide the **Database ID** for your Astra DB instance (see [here](https://awesome-astra.github.io/docs/pages/astra/faq/#where-should-i-find-a-database-identifier) for details);
- Ensure you have an **Access Token** for your database with role _Database Administrator_ (see [here](https://awesome-astra.github.io/docs/pages/astra/create-token/) for details).

Likewise, you will need the credentials to your Amazon Web Services identity, with access to **Amazon Bedrock**.

## Set up your Python environment

In [2]:
%pip install --quiet \
  "cassio>=0.1.3" \
  "langchain==0.0.249" \
  "boto3==1.28.62" \
  "botocore==1.31.62" \
  "cohere==4.37" \
  "openai==1.3.7" \
  "tiktoken==0.5.2" \
  "awscli==1.29.62"

Note: you may need to restart the kernel to use updated packages.


## Import needed libraries

In [1]:
import json
import os
import sys
from getpass import getpass


import boto3
import cassio

from langchain.embeddings import BedrockEmbeddings
from langchain.llms import Bedrock
from langchain.vectorstores import Cassandra
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader

## Astra DB Setup

In [ ]:
ASTRA_DB_ID = input("Enter your Astra DB ID ('0123abcd-'):")
ASTRA_DB_APPLICATION_TOKEN = getpass("Enter your Astra DB Token ('AstraCS:...'):")
ASTRA_DB_KEYSPACE = input("Enter your keyspace name (optional, default keyspace used if not provided):")

In [4]:
cassio.init(
    token=ASTRA_DB_APPLICATION_TOKEN,
    database_id=ASTRA_DB_ID,
    keyspace=ASTRA_DB_KEYSPACE if ASTRA_DB_KEYSPACE else None,
)

## AWS Credentials Setup

_Note_: in the following cells you will be asked to explicitly provide the credentials to your AWS account. These are set as environment variables for usage by the subsequent `boto3` calls. Please refer to [boto3's documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html) on the possible ways to supply your credentials in a more production-like environment.

In particular, if you are running this notebook in **Amazon SageMaker Studio**, please note that it is sufficient to add the Bedrock policy to your SageMaker role, as outlined at [this link](https://github.com/aws-samples/amazon-bedrock-workshop#enable-aws-iam-permissions-for-bedrock), to access the Bedrock services. In that case you can skip the following three setup cells.

I was able to make role work so i didn't need to set next three cells.

In [ ]:
# Input your AWS Access Key ID
os.environ["AWS_ACCESS_KEY_ID"] = getpass("Your AWS Access Key ID:")

In [ ]:
# Input your AWS Secret Access Key
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass("Your AWS Secret Access Key:")

In [ ]:
# Input your AWS Session Token
os.environ["AWS_SESSION_TOKEN"] = getpass("Your AWS Session Token:")

## Set up AWS Bedrock objects

In [6]:
bedrock_runtime = boto3.client("bedrock-runtime", "us-east-1")
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                                       client=bedrock_runtime)

## Set up the Vector Store

This command will create a suitable table in your database if it does not exist yet:

In [7]:
vector_store = Cassandra(
    embedding=bedrock_embeddings,
    table_name="shakespeare_act5",
    session=None,  # <-- meaning: use the global defaults from cassio.init()
    keyspace=None,  # <-- meaning: use the global defaults from cassio.init()
)

## Populate the database

Add lines for the text of "Romeo and Astra", Scene 5, Act 3

In [8]:
# retrieve the text of a scene from act 5 of Romeo and Astra.
# Juliet's name was changed to Astra to prevent the LLM from "cheating" when providing an answer.
! mkdir -p "texts"
! curl "https://raw.githubusercontent.com/awesome-astra/docs/main/docs/pages/aiml/aws/bedrock_resources/romeo_astra.json" \
    --output "texts/romeo_astra.json"
input_lines = json.load(open("texts/romeo_astra.json"))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75985  100 75985    0     0   345k      0 --:--:-- --:--:-- --:--:--  346k


Next, you'll populate the database with the lines from the play.
This can take a couple of minutes, please be patient.  In total there are 321 lines.


In [9]:
input_documents = []

for input_line in input_lines:
    if (input_line["ActSceneLine"] != ""):
        (act, scene, line) = input_line["ActSceneLine"].split(".")
        location = "Act {}, Scene {}, Line {}".format(act, scene, line)
        metadata = {"act": act, "scene": scene, "line": line}
    else:
        location = ""
        metadata = {}
    quote_input = "{} : {} : {}".format(location, input_line["Player"], input_line["PlayerLine"])
    input_document = Document(page_content=quote_input, metadata=metadata)
    input_documents.append(input_document)

print(f"Adding {len(input_documents)} documents ... ", end="")
vector_store.add_documents(documents=input_documents, batch_size=50)
print("Done.")

Adding 321 documents ... Done.


## Answer questions

In [10]:
prompt_template_str = """Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context

Question: {question}

Assistant:"""

prompt = PromptTemplate.from_template(prompt_template_str)

We choose to use the following LLM model (see [this page](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html#model-parameters-general) for more info):

In [11]:
model_id = "anthropic.claude-v2"

Here the question-answering function is set up, implementing the RAG pattern:

In [12]:
req_accept = "application/json"
req_content_type = "application/json"

# This, created from the vector store, will fetch the top relevant documents given a text query
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

def answer_question(question: str, verbose: bool = False) -> str:
    if verbose:
        print(f"\n[answer_question] Question: {question}")
    # Retrieval of the most relevant stored documents from the vector store:
    context_docs = retriever.get_relevant_documents(question)
    context = "\n".join(doc.page_content for doc in context_docs)
    if verbose:
        print("\n[answer_question] Context:")
        print(context)
    # Filling the prompt template with the current values
    llm_prompt_str = prompt.format(
        question=question,
        context=context,
    )
    # Invocation of the Amazon Bedrock LLM for text completion -- ultimately obtaining the answer
    llm_body = json.dumps({"prompt": llm_prompt_str, "max_tokens_to_sample": 500})
    llm_response = bedrock_runtime.invoke_model(
        body=llm_body,
        modelId=model_id,
        accept=req_accept,
        contentType=req_content_type,
    )
    llm_response_body = json.loads(llm_response["body"].read())
    answer = llm_response_body["completion"].strip()
    if verbose:
        print(f"\n[answer_question] Answer: {answer}\n")
    return answer

In [13]:
my_answer = answer_question("Who dies in the story?")
print("=" * 60)
print(my_answer)

Based on the provided context, it seems that Astra and Romeo both die in the story. The watchman finds Astra dead in the vault, and Balthasar brings news to Romeo that Astra has died. Romeo then goes to the vault to die and lie with Astra. So the context indicates that both Astra and Romeo die.


In [16]:
my_answer = answer_question("Who is Astra?")
print("=" * 60)
print(my_answer)

Based on the context provided, Astra seems to be a woman that Romeo was in love with, who died. Lines like "For here lies Astra, and her beauty makes" and "As that of true and faithful Astra" suggest she was beautiful and Romeo's love interest. Other lines mention her death and pining for someone other than Tybalt, indicating she is a separate character from Juliet. However, without more context about the overall story and characters, I cannot conclusively say who Astra is. The provided information suggests she is a love interest of Romeo who died, but I do not have enough context to definitively identify her role and relationship to other characters.


Let's take a look at the RAG process piece-wise:

In [17]:
my_answer = answer_question("In what era does this story happen?", verbose=True)
print("=" * 60)
print(my_answer)


[answer_question] Question: In what era does this story happen?

[answer_question] Context:
Act 5, Scene 3, Line 18 : PARIS : The boy gives warning something doth approach.
Act 5, Scene 3, Line 232 : PRINCE : Bring forth the parties of suspicion.
Act 5, Scene 3, Line 289 : PRINCE : Where is the county's page, that raised the watch?
Act 5, Scene 3, Line 176 : First Watchman : [Within]  Lead, boy: which way?
Act 5, Scene 3, Line 192 : First Watchman : Hold him in safety, till the prince come hither.

[answer_question] Answer: Based on the context provided, I do not have enough information to determine the era or time period that this story takes place in. The dialogue mentions a prince and pages, but without more contextual clues about the setting, technology, culture, etc., I cannot confidently identify the era.

Based on the context provided, I do not have enough information to determine the era or time period that this story takes place in. The dialogue mentions a prince and pages, b

### Interactive QA session

In [15]:
user_question = ""
while True:
    user_question = input("Enter a question (empty to quit):").strip()
    if user_question:
        print(f"Answer ==> {answer_question(user_question)}")
    else:
        print("[User, AI exeunt]")
        break

Enter a question (empty to quit): tell me learning from the story


Answer ==> Based on the provided context, I don't have enough information to summarize the key learnings from the full story. The lines provided are fragmented and don't provide a clear picture of the overall narrative or themes. Without more context, I cannot confidently identify the main takeaways or lessons.


Enter a question (empty to quit): who is the main character


Answer ==> Based on the provided context, I don't have enough information to determine the main character. The lines are spoken by multiple characters (Page, Friar Laurence, First Watchman, Balthasar) without indicating who the protagonist is. Without more context about the overall story and characters, I don't know who the main character is.


Enter a question (empty to quit): 


[User, AI exeunt]


## Additional resources

To learn more about Amazon Bedrock, visit this page: [Introduction to Amazon Bedrock](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/introduction-to-bedrock).